## Manufacturer Analysis using machine learning
The goal of this project is to find the most influential features from manufacturer profile using machine learning models such as RF, linear regression and Naisebase 

In [55]:
#Use match_helper.py file to join the SQL table
import pandas as pd
import math
import match_helper as helper
from imp import reload
reload(helper)
dbaccess = helper.get_dbaccess()
buyer_access = dbaccess + ['buyer']
manufacturer_access = dbaccess + ['manufacturer']
shared_access = dbaccess + ['shared']
crm_access = dbaccess + ['crm']
logging_access = dbaccess + ['logging']
messaging_access = dbaccess + ['messaging']
auth_access = dbaccess + ['auth']

/Users/glenhsu/Desktop/Foursource/Manufacturer analysis/


### Extract manufacturer features
The main features are extracted from Country - Fill Out Detail Tableau. (database: manufacturer)  


In [56]:
### Paste the photo of data structure here

In [86]:
# Create manufacturer table m_df (database: manufacturer)
reload(helper)
manufacturer = """
With manufacturer AS (SELECT
          m.id_manufacturer
        , m.name AS manufacturer_name
        , CASE WHEN m.profile_image != '' THEN 'yes' ELSE 'no' END AS has_profile_image
        , m.city AS manufacturer_city
        , MIN(country.name) AS manufacturer_country
        , CASE WHEN m.fk_manufacturer_status = 1 THEN 'Active' ELSE 'Deactivated' END AS manufacturer_status
        , m.created_at AS manufacturer_created_at
        , to_char(m.created_at::timestamp WITH TIME ZONE, 'iyyy"W"IW') AS m_created_year_week
        , to_char(m.created_at::timestamp WITH TIME ZONE, 'yyyy"M"mm') AS m_created_year_month
        , to_char(m.created_at::timestamp WITH TIME ZONE, 'yyyy') AS m_created_year
        , CASE 
                WHEN COUNT(DISTINCT(mt.name)) > 1 
                        THEN string_agg(DISTINCT(mt.name), ', ') 
                ELSE string_agg(DISTINCT(mt.name), '') 
          END AS manufacturer_types
        , CASE WHEN string_agg(DISTINCT(mt.name), '') IS NOT NULL THEN 'yes' ELSE 'no' END AS manufacturer_type_filled
        , CASE 
                WHEN COUNT(DISTINCT(c.id_contact)) > 1
                        THEN string_agg(DISTINCT(c.first_name || ' ' || c.last_name), ', ')
                ELSE string_agg(DISTINCT(c.first_name || ' ' || c.last_name), '')
          END AS all_contacts
        , COUNT(DISTINCT(c.id_contact)) AS contact_count
        , CASE
                WHEN COUNT(DISTINCT(c.id_contact)) > 1
                        THEN string_agg(DISTINCT(c.email), ', ')
                ELSE string_agg(DISTINCT(c.email), '')
          END AS emails
        , pf.filled_data
        , (pf.filled_data ->> 'all')::jsonb ->> 'percentage' AS all_fields_percentage
        , (pf.filled_data ->> 'mandatory')::jsonb ->> 'percentage' AS mandatory_fields_percentage
        , pf.filled_mandatory_reached
        , pf.filled_mandatory_reached_at
        , MIN(po.min_qty) AS po_min_qty
        , MIN(po.avg_qty) AS po_avg_qty
        , MIN(po.max_qty) AS po_max_qty
        , COUNT(DISTINCT(s.fk_service_type)) AS count_parent_services
        , COUNT(DISTINCT(ms.fk_service)) AS count_serives
        , CASE
                WHEN COUNT(DISTINCT(ms.fk_service)) > 1
                        THEN string_agg(DISTINCT(s.name), ', ')
                ELSE string_agg(DISTINCT(s.name), '')
          END AS services
        , (
                SELECT 
                        lt.minimum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 1
          ) AS on_stock_min
        , (
                SELECT 
                        lt.maximum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 1
          ) AS on_stock_max
        , CASE 
                WHEN (
                        SELECT 
                                lt.minimum + lt.maximum
                        FROM lead_time AS lt 
                        WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 1
                ) IS NOT NULL
                THEN 'yes'
                ELSE 'no'
          END AS on_stock_filled
        , (
                SELECT 
                        lt.minimum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 2
          ) AS made_to_order_min
        , (
                SELECT 
                        lt.maximum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 2
          ) AS made_to_order_max  
        , CASE 
                WHEN (
                        SELECT 
                                lt.minimum + lt.maximum
                        FROM lead_time AS lt 
                        WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 2
                ) IS NOT NULL
                THEN 'yes'
                ELSE 'no'
          END AS made_to_order_filled
        , (
                SELECT 
                        lt.minimum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 3
          ) AS sample_prototype_min
        , (
                SELECT 
                        lt.maximum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 3
          ) AS sample_prototype_max 
        , string_agg(DISTINCT( '(' || lt.minimum || '-' || lt.maximum || '-' || ltt.title || ')' ), ' / ') AS lead_time_detail
        , CASE WHEN COUNT(DISTINCT l.id_location) > 0 THEN 'yes' ELSE 'no' END AS has_location
        , COUNT(DISTINCT l.id_location) AS location_count
FROM manufacturer AS m
LEFT JOIN profile_fill AS pf ON pf.fk_manufacturer = m.id_manufacturer
LEFT JOIN country ON country.id_country = m.fk_country
LEFT JOIN manufacturer_manufacturer_type AS mtr ON mtr.fk_manufacturer = m.id_manufacturer
LEFT JOIN manufacturer_type AS mt ON mt.id_manufacturer_type = mtr.fk_manufacturer_type
LEFT JOIN manufacturer_user AS mu ON mu.fk_manufacturer = m.id_manufacturer
LEFT JOIN contact AS c ON c.id_contact = mu.fk_contact
LEFT JOIN purchase_order_size AS po ON po.fk_manufacturer = m.id_manufacturer
LEFT JOIN lead_time AS lt ON lt.fk_manufacturer = m.id_manufacturer
LEFT JOIN lead_time_type AS ltt ON ltt.id_lead_time_type = lt.fk_lead_time_type
LEFT JOIN manufacturer_service AS ms ON ms.fk_manufacturer = m.id_manufacturer
LEFT JOIN service AS s ON s.id_service = ms.fk_service 
LEFT JOIN location AS l ON l.fk_manufacturer = m.id_manufacturer AND l.active = TRUE
GROUP BY m.id_manufacturer, pf.id_profile_fill
ORDER BY m.id_manufacturer )


SELECT id_manufacturer,
        manufacturer_name,
        has_profile_image,
        manufacturer_city,
        manufacturer_country,
        manufacturer_status,
        manufacturer_type_filled,
        m_created_year_week,
        m_created_year_month,
        m_created_year,
        contact_count,
        all_fields_percentage,
        mandatory_fields_percentage,
        filled_mandatory_reached,
        po_min_qty,
        po_avg_qty,
        po_max_qty,
        count_parent_services,
        count_serives,
        on_stock_min,
        on_stock_max,
        made_to_order_min,
        made_to_order_max,
        sample_prototype_min,
        sample_prototype_max,
        location_count AS factory_location_count
        
FROM manufacturer

"""
m_df = helper.query_retrieval(manufacturer, manufacturer_access)
m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,po_max_qty,count_parent_services,count_serives,on_stock_min,on_stock_max,made_to_order_min,made_to_order_max,sample_prototype_min,sample_prototype_max,factory_location_count
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,1000000.0,2,7,NaN,NaN,NaN,NaN,NaN,NaN,2
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,50000.0,4,28,45.0,60.0,60.0,90.0,15.0,30.0,1
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,400000.0,2,7,50.0,70.0,60.0,90.0,20.0,30.0,9
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,500000.0,4,27,47.0,59.0,80.0,90.0,5.0,9.0,1


In [88]:
# create m_product_range_count table (database: shared)
m_product = """
SELECT
          t.fk_manufacturer AS id_manufacturer
        , COUNT(DISTINCT t.fk_ct1 || '-' || t.fk_ct2 || '-' || t.fk_ct3) AS count_parent_product_ranges
        , SUM(t.sub_product_category_count) AS sub_product_category_count
FROM (
        SELECT
                  mo.fk_manufacturer
                , ct.fk_ct1
                , ct.fk_ct2
                , ct.fk_ct3
                , COUNT(mo.id_manufacturer_offering) AS sub_product_category_count
        FROM manufacturer_offering AS mo
        LEFT JOIN category_tree AS ct ON ct.id_category_tree = mo.fk_category_tree
        WHERE mo.active = TRUE
        GROUP BY mo.fk_manufacturer, ct.fk_ct1, ct.fk_ct2, ct.fk_ct3
        ORDER BY mo.fk_manufacturer
) AS t
GROUP BY t.fk_manufacturer
"""
m_product_df = helper.query_retrieval(m_product, shared_access)
m_product_df[:5]

,id_manufacturer,count_parent_product_ranges,sub_product_category_count
0,1,2,7.0
1,2,3,17.0
2,4,4,23.0
3,5,5,32.0
4,6,2,4.0


In [90]:
# Create m_brand_product_range_count table (database: shared)

m_brand = """
SELECT
         t.fk_manufacturer AS id_manufacturer
       , SUM(t.brand_count) AS brand_count
       , COUNT(DISTINCT fk_ct1 || '-' || fk_ct2 || '-' || fk_ct3) AS count_parent_brands
FROM (
        SELECT
                  fk_manufacturer
                , fk_ct1
                , fk_ct2
                , fk_ct3
                , COUNT(fk_brand) AS brand_count
        FROM manufacturer_customer AS mc
        WHERE mc.fk_brand IS NOT NULL AND active = TRUE
        GROUP BY fk_manufacturer, fk_ct1, fk_ct2, fk_ct3
        ORDER BY fk_manufacturer
) AS t
GROUP BY t.fk_manufacturer

"""
m_brand_df = helper.query_retrieval(m_brand, shared_access)
m_brand_df[:5]

,id_manufacturer,brand_count,count_parent_brands
0,1,1.0,1
1,2,19.0,3
2,4,9.0,4
3,5,40.0,5
4,6,3.0,2


#### External join m_product_df and m_brand_df on m_df table

In [94]:
new_m_df = pd.merge(m_df, m_brand_df, on='id_manufacturer', how = 'left')
new_m_df = pd.merge(new_m_df, m_product_df, on='id_manufacturer', how = 'left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,on_stock_max,made_to_order_min,made_to_order_max,sample_prototype_min,sample_prototype_max,factory_location_count,brand_count,count_parent_brands,count_parent_product_ranges,sub_product_category_count
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,2,1.0,1.0,2.0,7.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,60.0,60.0,90.0,15.0,30.0,1,19.0,3.0,3.0,17.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,70.0,60.0,90.0,20.0,30.0,9,9.0,4.0,4.0,23.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,59.0,80.0,90.0,5.0,9.0,1,40.0,5.0,5.0,32.0


#### External join material & material_type count  (share database)

In [96]:
# Create material and material type table and join together with new_m_df
m_material = """
---material & material_type count

SELECT fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT m.id_material) AS material_count,
        COUNT(DISTINCT m.material_type) AS material_type_count
        
FROM(
        SELECT id_material,
                m.name AS material_name,
                fk_material_type AS material_type,
                mt.name AS material_type_name
        FROM material AS m
        LEFT JOIN material_type AS mt ON m.fk_material_type = mt.id_material_type
        WHERE active = true AND verified = true
) AS m
LEFT JOIN manufacturer_material AS mm ON m.id_material = mm.fk_material
WHERE mm.active = true
GROUP BY fk_manufacturer;
"""
m_material_df = helper.query_retrieval(m_material, shared_access)
new_m_df = pd.merge(new_m_df, m_material_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,sample_prototype_max,factory_location_count,brand_count,count_parent_brands,count_parent_product_ranges,sub_product_category_count,material_count_x,material_type_count_x,material_count_y,material_type_count_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,NaN,2,1.0,1.0,2.0,7.0,2.0,1.0,2.0,1.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,30.0,1,19.0,3.0,3.0,17.0,3.0,1.0,3.0,1.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,30.0,9,9.0,4.0,4.0,23.0,12.0,1.0,12.0,1.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,9.0,1,40.0,5.0,5.0,32.0,22.0,2.0,22.0,2.0


In [98]:
# Create fabric and fabric type table and join together with new_m_df
m_fabric = """
---fabric & fabric_type count

SELECT fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT f.id_fabric_type) AS fabric_count,
        COUNT(DISTINCT f.fabric_type) AS fabric_type_count
        
FROM(
        SELECT id_fabric_type,
                f.name AS fabric_name,
                fk_fabric_type_group AS fabric_type,
                ft.name AS fabric_type_name
        FROM fabric_type AS f
        LEFT JOIN fabric_type_group AS ft ON f.fk_fabric_type_group = ft.id_fabric_type_group
        WHERE active = true AND verified = true
) AS f
LEFT JOIN manufacturer_fabric_type AS mft ON f.id_fabric_type = mft.fk_fabric_type
WHERE mft.active = true
GROUP BY fk_manufacturer;
"""
m_fabric_df = helper.query_retrieval(m_fabric, shared_access)
new_m_df = pd.merge(new_m_df, m_material_df, on = 'id_manufacturer', how='left')
new_m_df[:5]


,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,count_parent_product_ranges,sub_product_category_count,material_count_x,material_type_count_x,material_count_y,material_type_count_y,material_count_x,material_type_count_x,material_count_y,material_type_count_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,2.0,7.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,3.0,17.0,3.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,4.0,23.0,12.0,1.0,12.0,1.0,12.0,1.0,12.0,1.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,5.0,32.0,22.0,2.0,22.0,2.0,22.0,2.0,22.0,2.0


In [100]:
# Create treatment and treatment type table and join together with new_m_df
m_treatment = """
---treatment & treatment_type count

SELECT fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT f.id_fabric_type) AS fabric_count,
        COUNT(DISTINCT f.fabric_type) AS fabric_type_count
        
FROM(
        SELECT id_fabric_type,
                f.name AS fabric_name,
                fk_fabric_type_group AS fabric_type,
                ft.name AS fabric_type_name
        FROM fabric_type AS f
        LEFT JOIN fabric_type_group AS ft ON f.fk_fabric_type_group = ft.id_fabric_type_group
        WHERE active = true AND verified = true
) AS f
LEFT JOIN manufacturer_fabric_type AS mft ON f.id_fabric_type = mft.fk_fabric_type
WHERE mft.active = true
GROUP BY fk_manufacturer;
"""
m_treatment_df = helper.query_retrieval(m_treatment, shared_access)
new_m_df = pd.merge(new_m_df, m_treatment_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,material_count_y,material_type_count_y,material_count_x,material_type_count_x,material_count_y,material_type_count_y,fabric_count_x,fabric_type_count_x,fabric_count_y,fabric_type_count_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,2.0,1.0,2.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,3.0,1.0,3.0,1.0,3.0,1.0,19.0,2.0,19.0,2.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,12.0,1.0,12.0,1.0,12.0,1.0,20.0,1.0,20.0,1.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,22.0,2.0,22.0,2.0,22.0,2.0,34.0,4.0,34.0,4.0


#### External join Employees number, Revenue/Year, Pieces/Year, reference customer brand count  (manufacturer,shared database)

In [102]:
# About: Employees number, Revenue / Year, Pieces / Year from manufacturer/manufacturer_key_factor

m_key = """
SELECT s.id_manufacturer,
        MIN( CASE WHEN s.id_key_factor = 1 AND s.value != '' THEN s.value ELSE null END ) AS num_Employee,
        MIN( CASE WHEN s.id_key_factor = 2 AND s.value != '' THEN s.value ELSE null END ) AS revenue_per_year,
        MIN( CASE WHEN s.id_key_factor = 3 AND s.value != '' THEN s.value ELSE null END ) AS pieces_per_year
FROM(
        SELECT m.fk_manufacturer AS id_manufacturer,
                m.value,
                k.name,
                k.id_key_factor
        FROM manufacturer_key_factor AS m
        LEFT JOIN key_factor AS k ON m.fk_key_factor = k.id_key_factor
        ) AS s
GROUP BY s.id_manufacturer
ORDER BY s.id_manufacturer
"""
m_key_df = helper.query_retrieval(m_key, manufacturer_access)
new_m_df = pd.merge(new_m_df, m_key_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,fabric_count_x,fabric_type_count_x,fabric_count_y,fabric_type_count_y,num_employee_x,revenue_per_year_x,pieces_per_year_x,num_employee_y,revenue_per_year_y,pieces_per_year_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,60000,3000000089,4000000000,60000,3000000089,4000000000
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,19.0,2.0,19.0,2.0,900,None,3600000,900,None,3600000
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,20.0,1.0,20.0,1.0,20000,300000000,95000000,20000,300000000,95000000
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,34.0,4.0,34.0,4.0,150,80000000,18000000,150,80000000,18000000


In [104]:
# About: reference customer from shared/manufacturer_brand count

m_ref = """
SELECT m.fk_manufacturer AS id_manufacturer,
        COUNT(m.fk_brand) AS ref_brand_count
FROM manufacturer_brand AS m
LEFT JOIN brand AS b ON m.fk_brand = b.id_brand
WHERE m.active = true AND b.verified = true
GROUP BY m.fk_manufacturer
ORDER BY m.fk_manufacturer;
"""
m_ref_df = helper.query_retrieval(m_ref, shared_access)
new_m_df = pd.merge(new_m_df, m_ref_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,fabric_count_y,fabric_type_count_y,num_employee_x,revenue_per_year_x,pieces_per_year_x,num_employee_y,revenue_per_year_y,pieces_per_year_y,ref_brand_count_x,ref_brand_count_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,NaN,NaN,60000,3000000089,4000000000,60000,3000000089,4000000000,1.0,1.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,19.0,2.0,900,None,3600000,900,None,3600000,9.0,9.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,20.0,1.0,20000,300000000,95000000,20000,300000000,95000000,13.0,13.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,34.0,4.0,150,80000000,18000000,150,80000000,18000000,12.0,12.0


In [ ]:
# About: description

m_des = """
SELECT m.id_manufacturer AS id_manufacturer,
CASE WHEN m.description = '' THEN 'no' ELSE 'yes' END AS has_description
FROM manufacturer AS m
where m.description IS NOT NULL;
"""
m_des_df = helper.query_retrieval(m_des, manufacturer_access)
new_m_df = pd.merge(new_m_df, m_des_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

#### External join garment production and quality assurance table (manufacturer database)

In [ ]:
# Machinery: fabric production, garment production and quality assurance from manufacturer/machine and service count
machine_key = """
SELECT  fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT CASE WHEN fk_service_type = 1 THEN id_service ELSE null END) AS num_fabric_production,
        COUNT(DISTINCT CASE WHEN fk_service_type = 2 THEN id_service ELSE null END) AS num_garment_production,
        COUNT(DISTINCT CASE WHEN fk_service_type = 3 THEN id_service ELSE null END) AS num_qualityassurance_production
FROM machine AS m
LEFT JOIN 
(
        SELECT *
        FROM service AS s
        LEFT JOIN service_type AS st ON s.fk_service_type = st.id_service_type
        WHERE s.verified = true AND s.active = true
) AS t ON m.fk_service = t.id_service
GROUP BY fk_manufacturer
ORDER BY fk_manufacturer;
"""
machine_key_df = helper.query_retrieval(machine_key, manufacturer_access)
machine_key_df[:5]
new_m_df = pd.merge(new_m_df, machine_key_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

#### External join showroom_visable, website link status and certificate compliance count number

In [ ]:
# Manufacturer Showroom is visable or not
show_key = """
SELECT  
        E.fk_entity AS id_manufacturer,
        visable AS showroom_visable
FROM (
        SELECT *,
                CASE 
                     WHEN visible_to_all = 'true' THEN 'True' 
                     WHEN visible_to_watchlist = 'true' OR visible_to_customers = 'true' THEN 'True' 
                     ELSE 'False' 
                END AS visable
        FROM feed AS f
        LEFT JOIN feed_type AS ft ON f.fk_feed_type = ft.id_feed_type
        WHERE f.fk_feed_type = 1
) AS E
WHERE E.fk_entity_type = 2 AND E.fk_feed_status = 1;
"""
show_key_df = helper.query_retrieval(show_key, shared_access)
show_key_df[:25]
new_m_df = pd.merge(new_m_df, show_key_df, on = 'id_manufacturer', how='left')
new_m_df[:25]

In [77]:
# Manufacturer website link status
web_key = """

SELECT fk_manufacturer AS id_manufacturer,
        active AS web_status 
FROM website AS w

"""
web_key_df = helper.query_retrieval(web_key, manufacturer_access)
new_m_df = pd.merge(new_m_df, web_key_df, on = 'id_manufacturer', how='left')
new_m_df[:5]


,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,pieces_per_year,ref_brand_count,has_description,num_fabric_production,num_garment_production,num_qualityassurance_production,showroom_visable_x,web_status_x,showroom_visable_y,web_status_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,4000000000,1.0,yes,NaN,NaN,NaN,NaN,True,NaN,True
1,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,4000000000,1.0,yes,NaN,NaN,NaN,NaN,True,NaN,False
2,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,4000000000,1.0,yes,NaN,NaN,NaN,NaN,False,NaN,True
3,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,4000000000,1.0,yes,NaN,NaN,NaN,NaN,False,NaN,False
4,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,3600000,9.0,yes,0.0,4.0,0.0,True,True,True,True


In [80]:
# count certificate compliance number
compliance_key = """

SELECT 
        fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT mc.fk_manufacturer_compliance_type) AS num_compliance
FROM manufacturer_compliance AS mc
LEFT JOIN manufacturer_compliance_type AS mct ON mc.fk_manufacturer_compliance_type = mct.id_manufacturer_compliance_type
WHERE mc.active = True
GROUP BY fk_manufacturer;

"""
compliance_key_df = helper.query_retrieval(compliance_key, manufacturer_access)
new_m_df = pd.merge(new_m_df, compliance_key_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,has_description,num_fabric_production,num_garment_production,num_qualityassurance_production,showroom_visable_x,web_status_x,showroom_visable_y,web_status_y,num_compliance_x,num_compliance_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,True,NaN,True,1.0,1.0
1,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,True,NaN,False,1.0,1.0
2,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,False,NaN,True,1.0,1.0
3,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,False,NaN,False,1.0,1.0
4,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,yes,0.0,4.0,0.0,True,True,True,True,3.0,3.0


In [84]:
new_m_df

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,has_description,num_fabric_production,num_garment_production,num_qualityassurance_production,showroom_visable_x,web_status_x,showroom_visable_y,web_status_y,num_compliance_x,num_compliance_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,True,NaN,True,1.0,1.0
1,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,True,NaN,False,1.0,1.0
2,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,False,NaN,True,1.0,1.0
3,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,yes,NaN,NaN,NaN,NaN,False,NaN,False,1.0,1.0
4,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,yes,0.0,4.0,0.0,True,True,True,True,3.0,3.0
5,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,yes,0.0,4.0,0.0,True,True,True,True,3.0,3.0
6,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,yes,0.0,4.0,0.0,True,True,True,True,3.0,3.0
7,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,yes,0.0,4.0,0.0,True,True,True,True,3.0,3.0
8,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,yes,0.0,4.0,0.0,True,True,True,True,3.0,3.0
9,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,yes,0.0,4.0,0.0,True,True,True,True,3.0,3.0
